In [2]:
#This project is based on the onw shared on youtube channel Alex the Analyst: https://www.youtube.com/watch?v=HiOtQMcI5wg

In [66]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import smtplib
import time
import datetime

In [67]:
#Connect to website for datasource
url = "https://www.amazon.com/Funny-Data-Systems-Business-Analyst/dp/B07FNW9FGJ/ref=sr_1_1?keywords=funny+got+data+mis+data+systems+business+analyst+t-shirt&qid=1647793465&sprefix=funny+got+data%2Caps%2C327&sr=8-1"

In [71]:
def set_up_scraper(keyword_search):
    #This method mimics the use of the browser which avoid the issue as seen usuing beautiful soup where Amazon blocks the 'BOT'
    #Source of bellow code and explation (with some modifications I have done): https://medium.com/@jb.ranchana/web-scraping-with-selenium-in-python-amazon-search-result-part-1-f09c88090932
    import selenium
    from selenium import webdriver
    from selenium.webdriver.common.by import By
    from selenium.webdriver.support.ui import WebDriverWait 
    from selenium.webdriver.support import expected_conditions as EC
    driver_path = 'chromedriver.exe'
    web = 'https://www.amazon.com/'
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    driver = webdriver.Chrome(options = options, executable_path = driver_path)
    driver.get(web)
    #search bar id= "twotabsearchtextbox"
    #search button id= id="nav-search-submit-button"
    #Keyword to search0
    keyword = keyword_search
    #web element for searchbox
    search_box = driver.find_element(by = By.ID,value ="twotabsearchtextbox")
    #Type the search  word in the search box
    search_box.send_keys(keyword)
    #create web element for the search button
    search_button = driver.find_element(by = By.ID,value = "nav-search-submit-button")
    #click search button
    search_button.click()
    #wait page for loading
    driver.implicitly_wait(5)
    #Create empty lists for the products details
    product_name = list()
    product_price = list()
    product_ratings = list()
    product_ratings_num = list()
    product_link = list()
    items = WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.XPATH, '//div[contains(@class, "s-result-item s-asin")]')))
    for item in items:
        name_l = item.find_elements(by=By.XPATH,value = './/span[@class="a-size-base-plus a-color-base a-text-normal"]')
        if name_l != []:
            name = name_l[0].text
        else:
            name = ''
        product_name.append(name)
        whole_price = item.find_elements(by=By.XPATH,value = './/span[@class="a-price-whole"]')
        fraction_price = item.find_elements(by=By.XPATH,value = './/span[@class="a-price-fraction"]')
        if whole_price != [] and fraction_price != []:
            price = '.'.join([whole_price[0].text,fraction_price[0].text])
        else:
            price = 0
        product_price.append(price)
        ratings_box = item.find_elements(by=By.XPATH,value = './/div[@class="a-row a-size-small"]/span')
        if ratings_box != []:
            ratings = ratings_box[0].get_attribute('aria-label')
            ratings_num = ratings_box[1].get_attribute('aria-label')
        else:
            ratings,ratings_num = 0,0
        product_ratings.append(ratings)
        product_ratings_num.append(ratings_num)
        link_l = item.find_elements(by=By.XPATH,value = './/a[@class="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"]')
        if link_l != []:
            link = link_l[0].get_attribute("href")
        else:
            link = ''
        product_link.append(link)
    driver.quit()
    for p in range(len(product_price)):
        product_price[p] =  float(product_price[p])
        #Adjust the numerical values from string to float
        if isinstance(product_ratings_num[p],str):
            product_ratings_num[p] = float(product_ratings_num[p].replace(',','.'))
    return product_name,product_price,product_ratings,product_ratings_num,product_link

In [72]:
def create_csv(keyword_search):
    data = set_up_scraper(keyword_search)
    print(data)
    header = ['name','price','rating','num_of_ratings','link','date']
    import csv
    with open(f'amazon_web_scraper_data_{keyword_search}.csv','w',newline='',encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(header)
        for r in range(len(product_name)):
            row = list()
            for d in data:
                row.append(d[r]) 
            row.append(datetime.date.today())
            writer.writerow(row)

def update_csv(keyword_search):
    data = set_up_scraper(keyword_search)
    header = ['name','price','rating','num_of_ratings','link','date']
    import csv
    with open(f'amazon_web_scraper_data_{keyword_search}.csv','a+',newline='',encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(header)
        for r in range(len(product_name)):
            row = list()
            for d in data:
                row.append(d[r]) 
            row.append(datetime.date.today())
            writer.writerow(row)

In [73]:
create_csv('tolkien book')

C:\Users\felip\AppData\Local\Temp/ipykernel_6756/2508647137.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(options = options, executable_path = driver_path)


(['The Nature Of Middle-Earth', 'Pictures by J.R.R. Tolkien', 'J.R.R. Tolkien 4-Book Boxed Set: The Hobbit and The Lord of the Rings', 'The Histories of Middle Earth, Volumes 1-5', 'Tales from the Perilous Realm', 'The Great Tales Of Middle-Earth: The Children of Húrin, Beren and Lúthien, and The Fall of Gondolin', 'The Silmarillion', 'The Atlas of Middle-Earth (Revised Edition)', 'Unfinished Tales Of Númenor And Middle-Earth', 'A Middle-Earth Traveler: Sketches from Bag End to Mordor', 'The History Of Middle-Earth Boxed Set', 'The Letters of J.R.R. Tolkien', 'The Book of Lost Tales 1', 'Estuche Tolkien (El Hobbit + La Comunidad + Las Dos Torres +El Retorno del Rey)', 'Unfinished Tales', 'The Hobbit & The Lord of the Rings Boxed Set: Illustrated edition', 'The Fall of Gondolin', 'The Lord Of The Rings Boxed Set', 'The Art Of The Lord Of The Rings By J.r.r. Tolkien', 'The Return of the King: Book Three in the Lord of the Rings Trilogy', "An Encyclopedia of Tolkien: The History and Mytho

IndexError: list index out of range

In [74]:
df = pd.read_csv('amazon_web_scraper_data_tolkien book.csv')
df

,name,price,rating,num_of_ratings,link,date
0,The Nature Of Middle-Earth,18.99,4.8 out of 5 stars,756.000,https://www.amazon.com/Nature-Middle-earth-J-R...,2022-03-21
1,Pictures by J.R.R. Tolkien,17.73,4.9 out of 5 stars,215.000,https://www.amazon.com/Pictures-J-R-R-Tolkien-...,2022-03-21
2,J.R.R. Tolkien 4-Book Boxed Set: The Hobbit an...,14.74,4.8 out of 5 stars,23.055,https://www.amazon.com/J-R-R-Tolkien-4-Book-Bo...,2022-03-21
3,"The Histories of Middle Earth, Volumes 1-5",29.47,4.8 out of 5 stars,608.000,https://www.amazon.com/Histories-Middle-Earth-...,2022-03-21
4,Tales from the Perilous Realm,0.00,4.8 out of 5 stars,1.290,https://www.amazon.com/Tales-from-Perilous-Rea...,2022-03-21
5,The Great Tales Of Middle-Earth: The Children ...,40.35,4.9 out of 5 stars,1.235,https://www.amazon.com/Great-Tales-Middle-eart...,2022-03-21
6,The Silmarillion,0.00,4.8 out of 5 stars,11.073,https://www.amazon.com/The-Silmarillion-J-R-R-...,2022-03-21
7,The Atlas of Middle-Earth (Revised Edition),18.99,4.8 out of 5 stars,1.125,https://www.amazon.com/Atlas-Middle-Earth-Revi...,2022-03-21
8,Unfinished Tales Of Númenor And Middle-Earth,14.99,4.8 out of 5 stars,1.208,https://www.amazon.com/Unfinished-N%C3%BAmenor...,2022-03-21
9,A Middle-Earth Traveler: Sketches from Bag End...,15.99,4.9 out of 5 stars,1.125,https://www.amazon.com/Middle-earth-Traveler-S...,2022-03-21


In [58]:
def run_everyday()
    # In case you want to check it everyday
    while True:
        update_csv('tolkien book')
        time.sleep(86400)

86400